# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = 'ollama'

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = "llama3.2"
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

There might be a problem with your API key? Please visit the troubleshooting notebook!


In [3]:
# # A class to represent a Webpage

# # Some websites need you to use proper headers when fetching them:
# headers = {
#  "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
# }

# class Website:
#     """
#     A utility class to represent a Website that we have scraped, now with links
#     """

#     def __init__(self, url):
#         self.url = url
#         response = requests.get(url, headers=headers)
#         self.body = response.content
#         soup = BeautifulSoup(self.body, 'html.parser')
#         self.title = soup.title.string if soup.title else "No title found"
#         if soup.body:
#             for irrelevant in soup.body(["script", "style", "img", "input"]):
#                 irrelevant.decompose()
#             self.text = soup.body.get_text(separator="\n", strip=True)
#         else:
#             self.text = ""
#         links = [link.get('href') for link in soup.find_all('a')]
#         self.links = [link for link in links if link]

#     def get_contents(self):
#         return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links and error handling.
    """
    
    def __init__(self, url):
        self.url = url
        self.title = "No title found"
        self.text = ""
        self.links = []
        
        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()  # Raise an error for bad responses (4xx and 5xx)
            self.body = response.content
            soup = BeautifulSoup(self.body, 'html.parser')
            
            self.title = soup.title.string if soup.title else "No title found"
            
            if soup.body:
                for irrelevant in soup.body(["script", "style", "img", "input"]):
                    irrelevant.decompose()
                self.text = soup.body.get_text(separator="\n", strip=True)
            
            links = [link.get('href') for link in soup.find_all('a')]
            self.links = [link for link in links if link]
        
        except requests.exceptions.RequestException as e:
            self.error = f"Failed to fetch webpage: {e}"
            self.text = "Error loading page."
            
    def get_contents(self):
        if hasattr(self, 'error'):
            return self.error
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"


In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'ht

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [37]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [6]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [7]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/

In [8]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [9]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/sesame/csm-1b',
 '/google/gemma-3-27b-it',
 '/Qwen/QwQ-32B',
 '/deepseek-ai/DeepSeek-R1',
 '/RekaAI/reka-flash-3',
 '/models',
 '/spaces/sesame/csm-1b',
 '/spaces/Qwen/QwQ-32B-Demo',
 '/spaces/Wan-AI/Wan2.1',
 '/spaces/ASLP-lab/DiffRhythm',
 '/spaces/JeffreyXiang/TRELLIS',
 '/spaces',
 '/datasets/FreedomIntelligence/medical-o1-reasoning-SFT',
 '/datasets/open-r1/codeforces-cots',
 '/datasets/facebook/natural_reasoning',
 '/datasets/Congliu/Chinese-DeepSeek-R1-Distill-data-110k',
 '/datasets/open-r1/OpenR1-Math-220k',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',
 '/docs/diffuser

In [42]:
get_links("https://huggingface.co")

{'about page': 'https://huggingface.co/team',
 'company page': 'https://huggingface.co/company',
 'careers/jobs page': 'https://apply.workable.com/huggingface/',
 'blog': 'https://discuss.huggingface.co/c/blog',
 'research and news': 'https://status.huggingface.co/',
 'community links': 'https://discuss.huggingface.co',
 'product pages': ['https://huggingface.co/deepseek-ai/DeepSeek-R1',
  'https://huggingface.co/reka-ai/reka-flash-3',
  'https://huggingface.co/allenai/transformers',
  'https://huggingface.co/facebook/natural_reasoning'],
 'product spaces': ['https://huggingface.co/spaces/sesame/csm-1b',
  'https://huggingface.co/spaces/Qwen/QwQ-32B-Demo',
  'https://huggingface.co/spaces/Wan-AI/Wan2.1'],
 'product datasets': ['https://huggingface.co/datasets/FreedomIntelligence/medical-o1-reasoning-SFT',
  'https://huggingface.co/datasets/open-r1/OpenR1-Math-220k',
  'https://huggingface.co/datasets/facebook/natural_reasoning']}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [11]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [45]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'home page', 'url': 'https://huggingface.co'}, {'type': 'About page', 'url': 'https://brand.huggingface.co/'}, {'type': 'Company/Research', 'url': 'https://discuss.huggingface.co/c/company/19597'}, {'type': 'Careers/Jobs Page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'Press/Contact', 'url': 'https://press.huggingface.co/'}, {'type': 'GitHub Repository', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter Account', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn Company Page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models

In [12]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [13]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [48]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/brand'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'Blog/Publishers', 'url': 'https://discuss.huggingface.co'}, {'type': 'Research/Discover', 'url': 'https://status.huggingface.co'}, {'type': 'Jobs/Discord', 'url': 'https://join.discord.com/huggingface'}, {'type': 'GitHub Projects', 'url': 'https://github.com/huggingface'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nsesame/csm-1b\nUpdated\n1 day ago\n•\n2.84k\n•\n1.04k\ngoogle/gemma-3-27b-it\nUpdated\n5 days ago\n•\n241k\n•\n720\nQwen/QwQ-32B\nUpdated\n6 days ago\n•\n417k\n•\n2.3k\ndeepseek-ai/DeepSeek-R1\nUpdated\n22 days ago\n•\n1.9M\n•\n11.4k\nRekaAI/reka-flash-3\nUpdated\n4 days ago\n•\n3.01k\n•\n281\nBrowse 1M+ models\nSpaces\nRunning\non\nZero\n420\n420\nSesame CSM\n🌱\nConversational speech generati

In [14]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [15]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'All', 'url': 'https://huggingface.co/'}, {'type': 'About company', 'url': 'https://allenn.ai/'}, {'type': 'Careers/Joins', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'Pricing', 'url': 'https://pricings.huggingface.co/'}, {'type': 'Enterprise', 'url': 'https://huggingface.co/enterprise/'}, {'type': 'GitHub', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

[HuggingFace Logo]
====================================

[Cover Image of a conversational speech bubble or a 3D model]

Welcome to Hugging Face, the AI community building the future.

**About Us**

Hugging Face is a powerful platform for machine learning collaborations. Our mission is to make it easy for researchers and developers to explore, create, and share AI applications using cutting-edge models and datasets.

**What We Do**

We provide a collaborative environment for:

* Host and manage public models, datasets, and applications
* Accelerate machine learning with optimized inference endpoints and deployment on GPU and TPU instances
* Share knowledge through documentation, blog posts, and community engagement

**Our Community**

Join over 50,000 organizations using Hugging Face, including top tech companies like Meta, Amazon, Google, Intel, Microsoft, and Grammarly.

**Our Resources**

We offer:

* A vast library of state-of-the-art machine learning models and datasets
* Open-source stack including Transformers, Diffusers, Safetensors, and more
* Tutorials, documentation, and community support

**Join the Movement**

* Sign up for our platform to start exploring AI applications
* Visit our [GitHub page](http://github.com/huggingface/) for the latest code updates
* Follow us on social media to stay updated on latest news and releases

**Careers**

If you're passionate about AI and want to join our community, explore our [Careers page](https://huggingface.co/jobs).

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [16]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [17]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/team'}, {'type': 'company web page', 'url': 'https://huggingface.co/'}, {'type': ' Careers/Jobs pages', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'Company overview and background', 'url': '/brand'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter handle', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn company profile', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Blog', 'url': 'https://discuss.huggingface.co'}, {'type': 'Documentation homepage', 'url': 'docs'}, {'type': 'Enterprise and pricing information', 'url': '/enterprise'}]}


**Hugging Face: Unlocking the Potential of Artificial Intelligence**

Welcome to Hugging Face, the AI community building the future. Our platform is a collaborative hub where machine learning professionals from around the world come together to create, discover, and apply models, datasets, and applications.

**About Us**

At Hugging Face, we are driven by a passion for artificial intelligence and its potential to transform industries and improve lives. Our team of experts is dedicated to building a foundation for ML tooling with the community, featuring state-of-the-art libraries and tools such as Transformers, Diffusers, Safetensors, and more.

**What We Do**

* Provide a platform for collaboration on models, datasets, and applications
* Offer access to 1M+ models, including cutting-edge models like sesame/csm-1b and QwQ 32B
* Enable users to create, discover, and apply AI apps in various modalities, such as text, image, video, audio, or even 3D
* Offer paid Compute and Enterprise solutions for accelerating ML development

**Community**

We strive to be inclusive of the entire machine learning community. Our platform is used by over 50,000 organizations, including:

* AI2 (Enterprise)
* Meta (Enterprise)
* Amazon (Company)
* Google (Company)
* Intel (Company)
* Microsoft (Company)
* Grammarly (Enterprise)
* Writer (Enterprise)

**Employment Opportunities**

If you're passionate about artificial intelligence and want to join our team, check out our jobs page for current openings. We offer a dynamic work environment with opportunities for growth and development.

**Getting Started**

Sign up on our platform or explore our tutorials and documentation to get started on your AI journey today!

---

Here is the brochure content in  format:

# [Hugging Face: Unlocking the Potential of Artificial Intelligence](#)

## About Us
----------------

At Hugging Face, we are driven by a passion for artificial intelligence and its potential to transform industries and improve lives.

## What We Do
-------------

* Provide a platform for collaboration on models, datasets, and applications
* Offer access to 1M+ models, including cutting-edge models like sesame/csm-1b and QwQ 32B
* Enable users to create, discover, and apply AI apps in various modalities, such as text, image, video, audio, or even 3D
* Offer paid Compute and Enterprise solutions for accelerating ML development

## Community
-------------

We strive to be inclusive of the entire machine learning community. Our platform is used by over 50,000 organizations, including:

* AI2 (Enterprise)
* Meta (Enterprise)
* Amazon (Company)
* Google (Company)
* Intel (Company)
* Microsoft (Company)
* Grammarly (Enterprise)
* Writer (Enterprise)

## Employment Opportunities
------------------------

If you're passionate about artificial intelligence and want to join our team, check out our jobs page for current openings. We offer a dynamic work environment with opportunities for growth and development.

## Getting Started
---------------

Sign up on our platform or explore our tutorials and documentation to get started on your AI journey today!

---

![Hugging Face Logo](https://huggingface.co/favicon.ico)

**Website:** [www.huggingface.com](http://www.huggingface.com)

**Social Media:**

* Twitter: [@huggingface](https://twitter.com/huggingface)
* LinkedIn: [The AI Community | Hugging Face](https://www.linkedin.com/company/10741785/)
* Discord: [Hugging Face - Join our community!](https://discord.com/invite/huggingface)

**Blog:** [The Hugging Face Blog](https://huggingface.co/blog)

**GitHub:** [Hugging Face Hub | GitHub](https://github.com/huggingface)

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>